<a href="https://colab.research.google.com/github/Shatokua/sent_analysis_dataset/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализ тональности по отношению к выбранному объекту

Конечной целью исследования является отладка модели для определения тональности текста по отношению к выбранному объекту текста, например, к упомянутой в тексте персоне, организации и т.д. 

Такой анализ является более точным, чем стандартный анализ тональности, и представляет больший интерес для конечных пользователей, поскольку дает оценку отношения к конкретному объекту, а не ко всему тексту.

## Подготовка окружения

In [ ]:
%%capture
#@title Установка окружения

!pip install -q sklearn==0.22.2.post1
!pip install mendelai-brat-parser==0.0.4
!pip install smart_open==5.1.0
!pip install tensorflow-text==2.5.0


In [ ]:
%%capture
#@title Импорт библиотек

import numpy as np
import pandas as pd
import os

from shutil import copyfile
from brat_parser import get_entities_relations_attributes_groups

from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  # для загрузки universal-sentence-encoder-cmlm/multilingual-preprocess
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
%%capture
#@title Определение рабочих директорий с данными
SOURCE_DIR1 = 'done/'
SOURCE_DIR2 = 'done1/'
BASE_DIR = 'train_test/'
main_csv_file = 'sent_quotes_done.csv'

## Изучение и подготовка данных

Данные представлены в виде текстовых файлов и файлов аннотаций в формате BRAT.
Все файлы находятся в двух папках done и done1. Файлы внутри папок не разделены по классам. Если в одном файле упоминается несколько объектов, по отношению к которым определяется тональность, данные находятся в одном файле ann.

Для работы в Tensorflow датасет может быть представлен в двух форматах:
1) как структурированный файл (например, csv), где каждая колонка является либо признаком, либо меткой класса;
2) набор файлов, распределенных по директориям-классам. 

Кроме того, необходимо создать копии

Подготовим данные для работы с Tensorflow
(**для последующей работы с Google Colab в дальнейшем будет использоваться сохраненный файл csv, исполнение следующих двух ячеек не требуется**).

In [ ]:
%%capture
#@title Функция для перевода данных в читаемый TensorFlow формат

def files_to_df(source_dir, target_dir):
    """Собираем список файлов txt. 
    Идём по списку файлов, для каждого файла извлекаем текст сообщения (из txt), 
    текст объекта (ann), его индексы (ann), тональность (ann). 
    Раскладываем файлы на папки по классам, параллельно записываем в 
    dataframe pandas"""

    if (target_dir[:-1] not in os.listdir()):
      os.mkdir(target_dir)

    file_names = [fn[:-4] for fn in os.listdir(source_dir) if fn[-3:]=='txt']
    
    text_to_process = []
    
    for fn in file_names:
        txt_file_path = source_dir + fn +'.txt'
        
        with open(txt_file_path, encoding="utf8") as f:
            txt = f.read()
        
        ann_file_path = source_dir + fn +'.ann'
        entities, relations, attributes, groups = get_entities_relations_attributes_groups(ann_file_path)
        entities_keys = list(entities.keys())
        
        for key in entities_keys:
            class_dir = entities[key].type
            if (class_dir not in os.listdir(target_dir)):
                os.mkdir(target_dir + class_dir)
                
            entity_id = entities[key].id

            text_to_process.append({'filename':fn+'_'+entity_id, 'text': txt, 'entity_id': entities[key].id, 
                                    'entity_text': entities[key].text, 'entity_span': entities[key].span[0],
                                   'label': entities[key].type})
            
            txt_new_file_path = target_dir + class_dir + '/' + fn + '_' + entity_id +'.txt'
            copyfile(txt_file_path, txt_new_file_path)
            ent_new_file_path = target_dir + class_dir + '/' + fn + '_' + entity_id + '_entity' +'.txt'
            with open(ent_new_file_path, mode ='w', encoding ="utf8") as f:
              f.write(entities[key].text)
    
    df = pd.DataFrame.from_dict(text_to_process)
    return df


In [ ]:
%%capture
#@title Изменение структуры папок, перенос данных в csv

df1 = files_to_df(SOURCE_DIR1, BASE_DIR)
df2 = files_to_df(SOURCE_DIR2, BASE_DIR)
df = pd.concat([df1, df2])
df.to_csv(main_csv_file)

Для работы в Google Colab требуется загрузка csv-файла.

In [ ]:
%%capture
#@title Чтение файл csv

csv_file = main_csv_file
dataframe = pd.read_csv(csv_file)

Изучим распределение датасета по классам

In [ ]:
%%capture
#@title Распределение классов


In [ ]:
print(dataframe.value_counts('label'))

label
Neutral_all     29070
Neutral         10354
Negative         4836
Positive         4657
Negative_all     1782
Positive_all      923
Mixed             291
Mixed_all          15
dtype: int64


Классы несбалансированы. Класс Mixed_all состоит всего из 15 экземпляров. Укрупним классы, объединив классы с постфиксом _all с одноименными без префикса.
Также приведем метки классов к числовым категориям

In [ ]:
%%capture
#@title Объединение классов, приведение меток классов к категориальному формату

classes = {'Positive': 0, 'Positive_all': 0, 'Negative': 1, 'Negative_all': 1,  
           'Mixed': 2, 'Mixed_all': 2, 'Neutral': 3, 'Neutral_all': 3}
dataframe['label'] = dataframe['label'].apply(lambda x: classes[x])

Разделим датасет на набор данных для тренировки и для тестирования.
В тренировочном датасете выделим набор для валидации

In [ ]:
%%capture
#@title Объединение классов, приведение меток классов к категориальному формату

train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)


In [ ]:
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

33233 train examples
8309 validation examples
10386 test examples


Поскольку классы не сбалансированы, проверим распределение по классам внутри тренировочного, тестового и валидационного датасетов

In [ ]:
%%capture
#@title Проверка распределения классов


In [ ]:
print('Нормализованное распределение по классам в начальном датасете')
print(dataframe.value_counts('label', normalize=True))

print('Нормализованное распределение по классам в датасете train')
print(train.value_counts('label', normalize=True))

print('Нормализованное распределение по классам в датасете test')
print(test.value_counts('label', normalize=True))

print('Нормализованное распределение по классам в датасете val')
print(val.value_counts('label', normalize=True))

Нормализованное распределение по классам в начальном датасете
label
3    0.759205
1    0.127446
0    0.107456
2    0.005893
dtype: float64
Нормализованное распределение по классам в датасете train
label
3    0.757049
1    0.127644
0    0.109439
2    0.005868
dtype: float64
Нормализованное распределение по классам в датасете test
label
3    0.764106
1    0.126805
0    0.103216
2    0.005873
dtype: float64
Нормализованное распределение по классам в датасете val
label
3    0.761704
1    0.127452
0    0.104826
2    0.006018
dtype: float64


Распределение классов одинаково во всех наборах.

## Выбор модели энкодера


Для работы выбрана модель [LaBSE](https://tfhub.dev/google/LaBSE/2) (Language-agnostic BERT sentence embedding model) как одна из наиболее актуальных моделей, показывающих хорошие результаты для русского языка.

Данные для этого энкодера должны быть предварительно обработаны препроцессором [universal-sentence-encoder-cmlm/multilingual-preprocess](https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2)

In [ ]:
labse_preprocessor = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
labse_encoder = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2")

## Изменение стандартного препроцессора

Стандартный препроцессор получает на вход текст и выдает в качестве output словарь из трех тензоров:  
- `input_word_ids`: id поданных на вход слов  
- `input_mask`: маска из 1 и 0, где 1 находится на позициях значимых слов, 0 на позициях паддинга  
- `input_type_ids`: маска из 0 и 1 для передачи дополнительной информации о токенах

Нам необходимо изменить препроцессор таким образом, что в маске `input_type_ids` на позициях токенов интересующего нас объекта стояли 1, на всех остальных позициях - 0.

In [ ]:
%%capture
#@title Вспомогательные функции 

"""
Вспомогательные функции для поиска в NumPy-массиве input_word_ids, 
соответствующих input_word_ids объекта
"""

def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
           
def findFirst_numpy(a, b):
    temp = rolling_window(a, len(b))
    result = np.where(np.all(temp == b, axis=1))
    return result[0][0] if result else None

In [ ]:
%%capture
#@title Функция для изменения массива type_id


def change_typeid(text_preprocessed, token_text, preprocessor): 
  """
  Принимает на вход результаты обработки строки препроцессором BERT
  (словарь с ключами input_word_ids, input_mask, input_type_ids)
  и текст токена.
  Меняет значения input_type_ids таким образом, что на позициях, соответствующих
  строке объекта token_text проставляются 1 вместо 0
  Возвращает маску из нолей и единиц для input_type_ids
  """
  token_preprocessed = preprocessor(token_text)
  token_numpy = token_preprocessed['input_word_ids'].numpy()[0]
  start = 1
  end = np.where(token_numpy == 102)[0][0]
  token_codes = token_numpy[start:end]

  text_numpy = text_preprocessed['input_word_ids'].numpy()[0]

  indx_start = findFirst_numpy(text_numpy, token_codes)
  indx_end = indx_start+len(token_codes)

  pattern_to_modify = text_preprocessed["input_type_ids"][0].numpy()
  pattern_to_modify[indx_start: indx_end] = 1
  pattern_to_modify = [pattern_to_modify]

  text_preprocessed["input_type_ids"] = tf.add (text_preprocessed["input_type_ids"], pattern_to_modify)
  return pattern_to_modify

In [ ]:
def custom_preprocess(texts, entities, preprocessor):
  inputs = preprocessor(texts)
  print(inputs)
  for i in range(inputs.shape[0]):
    change_typeid(inputs[i], entities[i], preprocessor)
  return inputs    

In [ ]:
text_test = ['"да, я допустил неточность, а посол Мюррей ссылается на слухи... но зато теперь мы накопали много РЕАЛЬНОЙ информации на Усманова', 'Мы тогда с сыном даже пять месяцев потом жили у Аллы Пугачевой. Порой она ночью садилась за рояль, а я ей говорил: "Тише. Максима разбудишь". Я все время искал с Алисой встречи. Помню, перелезал через забор в каком-то санатории. Регина делал все, чтобы мы отдалились. Это такой женский эгоизм» ']
token_text = ['Мюррей', 'Аллы Пугачевой']
text_preprocessed = custom_preprocess(text_test, token_text, labse_preprocessor)

{'input_type_ids': <tf.Tensor: shape=(2, 128), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>, 'input_word_ids': <tf.Tensor: shape=(2, 128),

KeyError: ignored

In [ ]:
text_preprocessed

{'input_mask': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
text_test = ['"да, я допустил неточность, а посол Мюррей ссылается на слухи... но зато теперь мы накопали много РЕАЛЬНОЙ информации на Усманова', 'Мы тогда с сыном даже пять месяцев потом жили у Аллы Пугачевой. Порой она ночью садилась за рояль, а я ей говорил: "Тише. Максима разбудишь". Я все время искал с Алисой встречи. Помню, перелезал через забор в каком-то санатории. Регина делал все, чтобы мы отдалились. Это такой женский эгоизм» ']
token_text = ['Мюррей', 'Аллы Пугачевой']
text_preprocessed = labse_preprocessor(text_test)
token_preprocessed = labse_preprocessor(token_text)
print(text_preprocessed)

{'input_type_ids': <tf.Tensor: shape=(2, 128), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>, 'input_word_ids': <tf.Tensor: shape=(2, 128),

In [ ]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
           
def findFirst_numpy(a, b):
    temp = rolling_window(a, len(b))
    result = np.where(np.all(temp == b, axis=1))
    return result[0][0] if result else None

In [ ]:
def change_typeid_arr(text_preprocessed, token_text, preprocessor):
  token_preprocessed = preprocessor(token_text)

  token_numpy = token_preprocessed['input_word_ids'].numpy()
  print (token_numpy)

  start = 1
  end = np.where(token_numpy == 102)
  print (end)


  text_numpy = text_preprocessed['input_word_ids'].numpy()


#  patterns = []
#  for j in range(len(text_numpy)):
#    indx_start = findFirst_numpy(text_numpy[j], token_codes[j])
#    indx_end = indx_start+len(token_codes[j])

#    pattern_to_modify = text_preprocessed["input_type_ids"][j].numpy()
#    pattern_to_modify[indx_start: indx_end] = 1
#    patterns.append(pattern_to_modify)
#  patterns = np.asarray(patterns)

#  text_preprocessed["input_type_ids"] = tf.add (text_preprocessed["input_type_ids"], patterns)



In [ ]:
print(f'Type Ids   : {text_preprocessed["input_type_ids"]}')

Type Ids   : [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [ ]:
change_typeid_arr(text_preprocessed, token_text, labse_preprocessor)

[[   101    677 162813 108855    102      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0]
 [   101  18856  17875    680  77545 380797    102      0      0      0
     

In [ ]:
print(f'Type Ids   : {text_preprocessed["input_type_ids"]}')

Type Ids   : [[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [ ]:
bert_results = encoder(text_preprocessed)


## Преобразование данных в датасет Tensorflow

In [ ]:
y_train = tf.keras.utils.to_categorical(train.label, num_classes=4)
y_val = tf.keras.utils.to_categorical(val.label, num_classes=4)
y_test = tf.keras.utils.to_categorical(test.label, num_classes=4)

In [ ]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Генерирует батчи данных

    Аргументы:
        texts_and_entities: массив текстов и объектов
        labels: массив меток
        batch_size: размер батча (int)
        shuffle: bool, перемешивать ли данные
        include_targets: bool, включать ли метки

    Возвращает:
        Tuple `([input_ids, attention_mask, `token_type_ids], labels)`
        (или только `[input_ids, attention_mask, `token_type_ids]`
         если `include_targets=False`)
    """

    def __init__(
        self,
        texts_and_entities,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.texts_and_entities = texts_and_entities
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        
        # Загружаем токенизатор BERT
        self.tokenizer =  hub.KerasLayer(preprocessor.tokenize)
        self.indexes = np.arange(len(self.texts_and_entities))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        texts_and_entities = self.texts_and_entities[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [ ]:
## В утиль
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('type')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

batch_size = 5
train_ds = df_to_dataset(train[['text', 'entity_text','type']], batch_size=batch_size)

[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of targets:', label_batch )
print('A batch of features (text):',train_features['text'])
print('A batch of features (entity_text):',train_features['entity_text'])

Every feature: ['text', 'entity_text']
A batch of targets: tf.Tensor([3 0 3 3 3], shape=(5,), dtype=int64)
A batch of features (text): tf.Tensor(
[b'\xd0\x9d\xd0\xbe \xd1\x82\xd0\xbe\xd0\xb3\xd0\xb4\xd0\xb0, \xd0\x9d\xd0\xb8\xd0\xba\xd0\xb8\xd1\x82\xd0\xb0 \xd0\xa1\xd0\xb5\xd1\x80\xd0\xb3\xd0\xb5\xd0\xb5\xd0\xb2\xd0\xb8\xd1\x87, \xd1\x8f \xd0\xbf\xd1\x80\xd0\xb8\xd0\xb7\xd1\x8b\xd0\xb2\xd0\xb0\xd1\x8e \xd0\xb2\xd0\xb0\xd1\x81 \xd0\xb1\xd1\x8b\xd1\x82\xd1\x8c \xd0\xbf\xd0\xbe\xd1\x81\xd0\xbb\xd0\xb5\xd0\xb4\xd0\xbe\xd0\xb2\xd0\xb0\xd1\x82\xd0\xb5\xd0\xbb\xd1\x8c\xd0\xbd\xd1\x8b\xd0\xbc. \xd0\x92\xd0\xb5\xd1\x80\xd0\xbd\xd0\xb8\xd1\x82\xd0\xb5 \xd0\xb2\xd0\xb0\xd1\x88\xd0\xb5\xd0\xb3\xd0\xbe \xd0\x9e\xd1\x81\xd0\xba\xd0\xb0\xd1\x80\xd0\xb0 \xd0\xbc\xd0\xb5\xd1\x80\xd0\xb7\xd0\xba\xd0\xb8\xd0\xbc \xd0\xb0\xd0\xbc\xd0\xb5\xd1\x80\xd0\xb8\xd0\xba\xd0\xb0\xd0\xbd\xd1\x86\xd0\xb0\xd0\xbc, \xd0\xba\xd0\xbe\xd1\x82\xd0\xbe\xd1\x80\xd1\x8b\xd0\xb5 "\xd0\xb1\xd0\xbe\xd0\xbc\xd0\xb1\xd1\x8f\xd1\x8

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = labse_preprocessor 
  encoder_inputs = preprocessing_layer(text_input)
  encoder = labse_encoder
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

tf.Tensor([[0.48670408]], shape=(1, 1), dtype=float32)


In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [ ]:
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

NameError: ignored

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

data = [
    "ξεῖν᾽, ἦ τοι μὲν ὄνειροι ἀμήχανοι ἀκριτόμυθοι",
    "γίγνοντ᾽, οὐδέ τι πάντα τελείεται ἀνθρώποισι.",
    "δοιαὶ γάρ τε πύλαι ἀμενηνῶν εἰσὶν ὀνείρων:",
    "αἱ μὲν γὰρ κεράεσσι τετεύχαται, αἱ δ᾽ ἐλέφαντι:",
    "τῶν οἳ μέν κ᾽ ἔλθωσι διὰ πριστοῦ ἐλέφαντος,",
    "οἵ ῥ᾽ ἐλεφαίρονται, ἔπε᾽ ἀκράαντα φέροντες:",
    "οἱ δὲ διὰ ξεστῶν κεράων ἔλθωσι θύραζε,",
    "οἵ ῥ᾽ ἔτυμα κραίνουσι, βροτῶν ὅτε κέν τις ἴδηται.",
]
layer = preprocessing.TextVectorization()
layer.adapt(data)
vectorized_text = layer(data)
print(vectorized_text)

StringLookup

tf.Tensor(
[[37 12 25  5  9 20 21  0  0]
 [51 34 27 33 29 18  0  0  0]
 [49 52 30 31 19 46 10  0  0]
 [ 7  5 50 43 28  7 47 17  0]
 [24 35 39 40  3  6 32 16  0]
 [ 4  2 15 14 22 23  0  0  0]
 [36 48  6 38 42  3 45  0  0]
 [ 4  2 13 41 53  8 44 26 11]], shape=(8, 9), dtype=int64)
